In [3]:
import pandas as pd 
from datetime import datetime
import re 

Importing the Data

In [ ]:
orders=pd.read_csv("../data/raw/Orders.csv", sep=';',usecols=range(5))
customers=pd.read_csv("../data/raw/Customers.csv", sep=';')
product=pd.read_csv("../data/raw/Product.csv", sep=';')

Checking for null values

In [3]:
orders.isna().sum()

Order ID       0
Order Date     0
Customer ID    0
Product ID     0
Quantity       0
dtype: int64

In [4]:
product.isna().sum()

Product ID        0
Coffee Type       0
Roast Type        0
Size              0
Unit Price        0
Price per 100g    0
Profit            0
dtype: int64

In [5]:
customers.isna().sum()

Customer ID         0
Customer Name       0
Email             204
Phone Number      130
Address Line 1      0
City                0
Country             0
Postcode            0
Loyalty Card        0
dtype: int64

--> Only the customers table has null values in the email and phone number columns

Orders table cleaning 

In [5]:
orders.columns = orders.columns.str.strip().str.lower().str.replace(" ", "_")
orders['order_date'] = pd.to_datetime(orders['order_date'],format='%d/%m/%Y') 
today = datetime.today().strftime('%d/%m/%Y')
orders = orders.groupby(['order_id', 'order_date', 'customer_id', 'product_id'], as_index=False)['quantity'].sum()

print(orders[orders['order_date'] > today]) 
print(orders.info())


Empty DataFrame
Columns: [order_id, order_date, customer_id, product_id, quantity]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     999 non-null    object        
 1   order_date   999 non-null    datetime64[ns]
 2   customer_id  999 non-null    object        
 3   product_id   999 non-null    object        
 4   quantity     999 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 39.2+ KB
None


In [17]:
orders[orders.duplicated(subset=['order_id','customer_id','product_id','quantity'], keep=False)]


,order_id,order_date,customer_id,product_id,quantity


In [23]:
orders.head(3)

,order_id,order_date,customer_id,product_id,quantity
0,AAQ-13644-699,2022-06-03,46296-42617-OQ,R-D-1,4
1,ABK-08091-531,2020-10-30,53864-36201-FG,L-L-1,3
2,ABO-29054-365,2019-01-19,00256-19905-YG,A-M-0.5,6


Products table cleaning

In [6]:
product.columns=product.columns.str.strip().str.lower().str.replace(" ", "_")
cols_to_float=['size','unit_price','price_per_100g','profit'] 
product[cols_to_float]=product[cols_to_float].apply(lambda x: x.str.replace(',', '.').astype(float))
print(product.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   product_id      48 non-null     object 
 1   coffee_type     48 non-null     object 
 2   roast_type      48 non-null     object 
 3   size            48 non-null     float64
 4   unit_price      48 non-null     float64
 5   price_per_100g  48 non-null     float64
 6   profit          48 non-null     float64
dtypes: float64(4), object(3)
memory usage: 2.8+ KB
None


In [14]:
product[product.duplicated(subset=['product_id'], keep=False)]


,product_id,coffee_type,roast_type,size,unit_price,price_per_100g,profit


In [22]:
product.head(3)

,product_id,coffee_type,roast_type,size,unit_price,price_per_100g,profit
0,A-L-0.2,Ara,L,0.2,3.885,1.9425,0.34965
1,A-L-0.5,Ara,L,0.5,7.770,1.5540,0.69930
2,A-L-1,Ara,L,1.0,12.950,1.2950,1.16550


Customers data cleaning

In [8]:
customers.columns=customers.columns.str.strip().str.lower().str.replace(" ", "_")
customers = customers.dropna(subset=['customer_id']) 
customers['country']=customers['country'].str.upper().replace({'UNITED STATES':'US','UNITED KINGDOM':'UK','IRELAND':'IR'})
customers.fillna({'customer_name':'No name','email':'No email','phone_number':'No phone number'},inplace=True)

def validate_phone_number(row): 
    phone_number = row['phone_number']
    country = row['country']
    if country == 'US':
        pattern = r'^\+1 \(\d{3}\) \d{3}-\d{4}$'
    elif country == 'UK':
        pattern = r'^\+44 \(\d{3}\) \d{3}-\d{4}$'
    elif country == 'IR':
        pattern = r'^\+353 \(\d{3}\) \d{3}-\d{4}$'
    else:
        return False
    return bool(re.match(pattern, phone_number))

customers['PhoneNumberValid'] = customers.apply(validate_phone_number, axis=1)

print(customers.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   customer_id       1000 non-null   object
 1   customer_name     1000 non-null   object
 2   email             1000 non-null   object
 3   phone_number      1000 non-null   object
 4   address_line_1    1000 non-null   object
 5   city              1000 non-null   object
 6   country           1000 non-null   object
 7   postcode          1000 non-null   object
 8   loyalty_card      1000 non-null   object
 9   PhoneNumberValid  1000 non-null   bool  
dtypes: bool(1), object(9)
memory usage: 71.4+ KB
None


In [13]:
customers[customers.duplicated(subset=['customer_id'], keep=False)]


,customer_id,customer_name,email,phone_number,address_line_1,city,country,postcode,loyalty_card,PhoneNumberValid


In [21]:
customers.head(3)

,customer_id,customer_name,email,phone_number,address_line_1,city,country,postcode,loyalty_card,PhoneNumberValid
0,17670-51384-MA,Aloisia Allner,aallner0@lulu.com,+1 (862) 817-0124,57999 Pepper Wood Alley,Paterson,US,7505,Yes,True
1,73342-18763-UW,Piotr Bote,pbote1@yelp.com,+353 (913) 396-4653,2112 Ridgeway Hill,Crumlin,IR,D6W,No,True
2,21125-22134-PX,Jami Redholes,jredholes2@tmall.com,+1 (210) 986-6806,5214 Bartillon Park,San Antonio,US,78205,Yes,True


Creating Time Dimension

In [18]:
time_dim = pd.DataFrame({
    'order_date': orders['order_date'],
    'day': orders['order_date'].dt.day_name(),                   # Extract the day
    'month_name': orders['order_date'].dt.month_name(),   # Extract the month name
    'year': orders['order_date'].dt.year                  # Extract the year
})

time_dim = time_dim.drop_duplicates(subset=['order_date'], keep='first')
time_dim.info()
time_dim.head()

<class 'pandas.core.frame.DataFrame'>
Index: 689 entries, 0 to 997
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   order_date  689 non-null    datetime64[ns]
 1   day         689 non-null    object        
 2   month_name  689 non-null    object        
 3   year        689 non-null    int32         
dtypes: datetime64[ns](1), int32(1), object(2)
memory usage: 24.2+ KB


,order_date,day,month_name,year
0,2022-06-03,Friday,June,2022
1,2020-10-30,Friday,October,2020
2,2019-01-19,Saturday,January,2019
3,2021-01-28,Thursday,January,2021
4,2021-07-24,Saturday,July,2021


Joining the data

In [20]:
full_df=orders.merge(customers,on='customer_id',how='inner') \
              .merge(product,on='product_id',how="inner") \
              .merge(time_dim,on='order_date',how="inner")

full_df.info()
full_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_id          999 non-null    object        
 1   order_date        999 non-null    datetime64[ns]
 2   customer_id       999 non-null    object        
 3   product_id        999 non-null    object        
 4   quantity          999 non-null    int64         
 5   customer_name     999 non-null    object        
 6   email             999 non-null    object        
 7   phone_number      999 non-null    object        
 8   address_line_1    999 non-null    object        
 9   city              999 non-null    object        
 10  country           999 non-null    object        
 11  postcode          999 non-null    object        
 12  loyalty_card      999 non-null    object        
 13  PhoneNumberValid  999 non-null    bool          
 14  coffee_type       999 non-

,order_id,order_date,customer_id,product_id,quantity,customer_name,email,phone_number,address_line_1,city,...,PhoneNumberValid,coffee_type,roast_type,size,unit_price,price_per_100g,profit,day,month_name,year
0,AAQ-13644-699,2022-06-03,46296-42617-OQ,R-D-1,4,Fernando Sulman,fsulmanl0@washington.edu,+1 (828) 464-2678,45 Village Terrace,Asheville,...,True,Rob,D,1.0,8.95,0.895,0.5370,Friday,June,2022
1,ABK-08091-531,2020-10-30,53864-36201-FG,L-L-1,3,Tess Benediktovich,tbenediktovichmv@ebay.com,+1 (505) 523-8113,1068 Sutherland Plaza,Albuquerque,...,True,Lib,L,1.0,15.85,1.585,2.0605,Friday,October,2020
2,ABO-29054-365,2019-01-19,00256-19905-YG,A-M-0.5,6,Stanislaus Valsler,No email,+353 (479) 865-9222,95 Southridge Alley,Castlebridge,...,True,Ara,M,0.5,6.75,1.350,0.6075,Saturday,January,2019


Saving the transfomed datasets

to csv

In [ ]:
orders.to_csv("../data/cleaned/order_cleaned.csv",index=False)
customers.to_csv("../data/cleaned/customers_cleaned.csv",index=False)
product.to_csv("../data/cleaned/product_cleaned.csv",index=False)
time_dim.to_csv("../data/cleaned/time_dim.csv",index=False)
full_df.to_csv("../data/joined/joined_data.csv",index=False)

to excel

In [ ]:
with pd.ExcelWriter('../data/joined/joined_data.xlsx') as writer:
    orders.to_excel(writer, sheet_name='Orders', index=False)
    customers.to_excel(writer, sheet_name='Customers', index=False)
    product.to_excel(writer, sheet_name='Products', index=False)
    time_dim.to_excel(writer,sheet_name='Time', index=False)